In [ ]:
from uniprotparser import parser
import pandas as pd
import io

In [ ]:
d = pd.read_csv(r"C:\Users\Toan Phung\Downloads\test_Copies_02.txt", sep="\t")
acc = set()
for a in d["PG.ProteinGroups"]:
    for i in a.split(";"):
        accession = parser.UniprotSequence(i.strip(), parse_acc=True)
        if accession.accession:
            acc.add(accession)

In [ ]:
p = parser.UniprotParser(acc, unique=True)
df = []
for res in p.parse("tab", "post"):
    df.append(pd.read_csv(io.StringIO(res), sep="\t"))

In [ ]:
df = df[0]
for i, r in d.iterrows():
    acc = r["Protein.Group"].split(";")
    for a in acc:
        ac = a.split("-")[0]
        df2 = df[df["Entry"] == ac]
        for i2, r2 in df2.iterrows():
            d.at[i, "Mol.wt"] = r2["Mass"]
            break

In [ ]:
d.to_csv(r"C:\Users\Toan Phung\Downloads\test_Copies_02.txt", index=False, sep="\t")